In [1]:
import torch 

torch.cuda.is_available()

True

In [2]:
import pandas as pd

dataset = pd.read_csv('data/labeled_data.csv')

In [3]:
allowed_values = ['AP', 'PA', 'LATERAL', 'LL']

dataset = dataset[dataset['ViewPosition'].isin(allowed_values)]

In [4]:
len(dataset)

361316

In [5]:
from sklearn.model_selection import train_test_split

train_dataset, test_dataset = train_test_split(dataset, test_size=.025, shuffle=True, random_state=42)
test_dataset, validation_dataset = train_test_split(test_dataset, test_size=.5, shuffle=True,random_state=42)

print(f'Len Train Dataset: {len(train_dataset)}')
print(f'Len Validation Dataset: {len(validation_dataset)}')
print(f'Len Test Dataset: {len(test_dataset)}')

Len Train Dataset: 352283
Len Validation Dataset: 4517
Len Test Dataset: 4516


In [6]:
from torch.utils.data import Dataset
from PIL import Image

class MedicalDataset(Dataset):
    def __init__(self, csv_dataset, transform=None):
        super(MedicalDataset, self).__init__()
        self.data = csv_dataset
        self.transform = transform
        
        self.label_map = {
            'AP': 0,
            'PA': 1,
            'LATERAL': 2,
            'LL': 3
        }
        
        self.data['ViewPosition'] = self.data['ViewPosition'].map(self.label_map)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        
        image_path = row['Image_Path']
        image = Image.open(image_path).convert('L')
        
        if self.transform:
            image = self.transform(image)
            
        image = image.to('cuda')
        
        label = torch.tensor(row['ViewPosition'], dtype=torch.long).to('cuda')
        
        return image, label

In [7]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

train_dataset = MedicalDataset(train_dataset, transform)
validation_dataset = MedicalDataset(validation_dataset, transform)
test_dataset = MedicalDataset(test_dataset, transform)

In [8]:
from torch.utils.data import DataLoader

train_dataset = DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_dataset = DataLoader(validation_dataset, batch_size=32, shuffle=True)
test_dataset = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [9]:
from torch import nn

class MedicalCNN(nn.Module):
    def __init__(self, num_classes=4):
        super(MedicalCNN, self).__init__()
        
        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv_2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv_3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        
        self.fully_connected_1 = nn.Linear(128 * 32 * 32, 512)
        self.fully_connected_2 = nn.Linear(512, num_classes)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=.5)
        
        
    def forward(self, x):
        x = self.pool(self.relu(self.conv_1(x)))
        x = self.pool(self.relu(self.conv_2(x)))
        x = self.pool(self.relu(self.conv_3(x)))
        
        x = x.view(-1, 128 * 32 * 32)
        x = self.relu(self.fully_connected_1(x))
        x = self.dropout(x)
        x = self.fully_connected_2(x)
        
        return x

In [10]:
from torch import optim

model = MedicalCNN(num_classes=4).to('cuda')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [11]:
from tqdm import tqdm

num_epochs = 1

for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    batch_count = 0

    for images, labels in tqdm(train_dataset, desc=f"Epoch {epoch+1}/{num_epochs}"):
        images, labels = images.to('cuda'), labels.to('cuda')
        model.train()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        batch_count += 1
        if batch_count % 313 == 0:
            model.eval()
            valid_loss = 0.0
            valid_correct = 0
            valid_total = 0
            
            with torch.no_grad():
                for valid_images, valid_labels in validation_dataset:
                    valid_images, valid_labels = valid_images.to('cuda'), valid_labels.to('cuda')
                    valid_outputs = model(valid_images)
                    valid_loss_item = criterion(valid_outputs, valid_labels)
                    
                    valid_loss += valid_loss_item.item()
                    _, valid_predicted = torch.max(valid_outputs, 1)
                    valid_total += valid_labels.size(0)
                    valid_correct += (valid_predicted == valid_labels).sum().item()

            valid_loss = valid_loss / len(validation_dataset)
            valid_accuracy = 100 * valid_correct / valid_total
            print(f"Batch {batch_count}: Train Loss: {running_loss / (batch_count)} | "
                  f"Train Accuracy: {100 * correct / total:.2f}% | "
                  f"Validation Loss: {valid_loss:.4f} | Validation Accuracy: {valid_accuracy:.2f}%")

    epoch_loss = running_loss / len(train_dataset)
    epoch_accuracy = 100 * correct / total
    print(f"Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_accuracy:.2f}%")

Epoch 1/1:   3%|▎         | 313/11009 [33:31<547:54:09, 184.41s/it]

Batch 313: Train Loss: 0.38581282192002087 | Train Accuracy: 85.16% | Validation Loss: 0.1714 | Validation Accuracy: 94.64%


Epoch 1/1:   6%|▌         | 626/11009 [58:43<179:46:15, 62.33s/it] 

Batch 626: Train Loss: 0.2764195495109541 | Train Accuracy: 89.99% | Validation Loss: 0.1244 | Validation Accuracy: 96.46%


Epoch 1/1:   9%|▊         | 939/11009 [1:24:53<259:38:47, 92.82s/it]

...

Batch 10642: Train Loss: 0.09363973015467937 | Train Accuracy: 97.40% | Validation Loss: 0.0576 | Validation Accuracy: 98.56%


Epoch 1/1: 100%|█████████▉| 10955/11009 [14:00:29<52:11, 57.99s/it]  

Batch 10955: Train Loss: 0.09240689024316885 | Train Accuracy: 97.44% | Validation Loss: 0.0631 | Validation Accuracy: 98.67%


Epoch 1/1: 100%|██████████| 11009/11009 [14:03:53<00:00,  4.60s/it]

Train Loss: 0.0923, Train Accuracy: 97.45%


In [12]:
torch.save(model.state_dict(), 'model_1.pth')

In [13]:
model.eval()
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_dataset:
        images, labels = images.to('cuda'), labels.to('cuda')
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        test_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_loss = test_loss / len(test_dataset)
test_accuracy = 100 * correct / total
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")